Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Azure Machine Learning Pipeline with AutoML (SDK v2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

In [ ]:
# 以前のプレビュー版がインストールされている場合にはアンインストールします
# !pip uninstall azure-ai-ml

In [ ]:
# 最新版の Python SDK v2 をインストールします
# !pip install azure-ai-ml

In [ ]:
# !pip install --upgrade azure-ai-ml

In [ ]:
# SDK バージョンを確認します。定期的に pip install --upgrade azure-ai-ml を実行し、最新に保つことを推奨します。

import azure.ai.ml
print("SDK version:", azure.ai.ml.VERSION)

## Connect to Azure Machine Learning Workspace

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AZUREML_WORKSPACE_NAME>"

In [ ]:
# クライアント認証を実施します。

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace)


In [ ]:
# ワークスペース情報の表示

print(ml_client.subscription_id)

for ws in ml_client.workspaces.list():
    print(ws.name, ws.resource_group, ws.location, sep = '\n')

## Create an Azure ML environment for pipeline

In [ ]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [ ]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
  - anaconda
dependencies:
  - python=3.8.5
  - pip:
    - azure-ai-ml
    - msal

In [ ]:
from azure.ai.ml.entities import Environment

custom_env_name = "sample-environment"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="sample environment",
    conda_file=os.path.join(dependencies_dir, "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

## Compute target setup

In [ ]:
# パイプラインの作成、実行するコンピューティング クラスターを作成します。既に存在する場合は作成せずスキップします。
from azure.ai.ml.entities import AmlCompute

cpu_compute_target = "cpu-cluster"

try:
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=180,
        tier="Dedicated",
    )

    ml_client.compute.begin_create_or_update(cpu_cluster)
    cpu_cluster = ml_client.compute.get(cpu_compute_target)

print(
    f"AMLCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)

## Prepare DataSet

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# トレーニング用のデータを読み込みます
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data"
)

## Create Pipeline

In [ ]:
%%writefile sdkv2_deployment.py
import time
import argparse
import datetime
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

parser = argparse.ArgumentParser()
parser.add_argument("--model_input_path", type=str, help="Path to input model")
parser.add_argument(
    "--model_base_name", type=str, help="Name of the registered model"
)
parser.add_argument("--subscription_id", type=str, help="Subscription id")
parser.add_argument("--resource_group", type=str, help="Resource group name")
parser.add_argument("--workspace", type=str, help="Workspace name")
args = parser.parse_args()

credential = AzureMLOnBehalfOfCredential()
ml_client = MLClient(credential, args.subscription_id, args.resource_group, args.workspace)

model_name = args.model_base_name

file_model = Model(
        path=args.model_input_path,
        type=AssetTypes.MLFLOW_MODEL,
        name=model_name,
        description="Model created from local file."
    )

ml_client.models.create_or_update(file_model)

latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=model_name)]
)

registered_model = ml_client.models.get(name=model_name, version=latest_model_version)
online_endpoint_name = "sample-endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="An online endpoint to generate predictions for the diabetes dataset",
    auth_mode="key",
    tags={"foo": "bar"},
)

create_endpoint = ml_client.begin_create_or_update(endpoint)

while create_endpoint.done() == False:
    time.sleep(10)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=registered_model,
    instance_type="Standard_F4s_v2",
    instance_count=1
)

deployment = ml_client.online_deployments.begin_create_or_update(blue_deployment)

while deployment.done() == False:
    time.sleep(10)

In [ ]:
from azure.ai.ml import automl, command, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import UserIdentityConfiguration


@pipeline(description="AutoML Regression and Deployment Pipeline")
def automl_regression_and_deployment(input_data, subscription_id, resource_group, workspace):
    

    automl_regression_node = automl.regression(
        training_data=input_data,
        target_column_name="ERP",
        primary_metric="R2Score",
        n_cross_validations=5,
        enable_model_explainability=True,
        tags={"my_custom_tag": "My custom value"},
        outputs={"best_model": Output(type="mlflow_model")}
    )

    automl_regression_node.set_limits(max_trials=1, max_concurrent_trials=1)
    automl_regression_node.set_training(
        enable_stack_ensemble=False, enable_vote_ensemble=False
    )


    deployment_command_func = command(
        inputs=dict(
            model_input_path=Input(type="mlflow_model"),
            model_base_name="regression_example_model",
            subscription_id=str(subscription_id),
            resource_group=str(resource_group),
            workspace=str(workspace)
        ),
        code="./sdkv2_deployment.py",
        command="python sdkv2_deployment.py "
        + "--model_input_path ${{inputs.model_input_path}} "
        + "--model_base_name ${{inputs.model_base_name}} "
        + "--subscription_id ${{inputs.subscription_id}} "
        + "--resource_group ${{inputs.resource_group}} "
        + "--workspace ${{inputs.workspace}}",
        environment="sample-environment@latest",
    )

    deployment_node = deployment_command_func(model_input_path=automl_regression_node.outputs.best_model)
    deployment_node.identity = UserIdentityConfiguration()


In [ ]:
pipeline_automl_regression_and_deployment = automl_regression_and_deployment(
    input_data=my_training_data_input,
    subscription_id=subscription_id,
    resource_group=resource_group,
    workspace=workspace
)

pipeline_automl_regression_and_deployment.settings.default_compute = "cpu-cluster"

## Run

In [ ]:
# パイプライン ジョブを送信します
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_automl_regression_and_deployment, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# ジョブの状態を表示します
ml_client.jobs.stream(pipeline_job.name)

## Inference

In [ ]:
# エンドポイントを表示します
for oe in ml_client.online_endpoints.list():
    print(oe.name)

In [ ]:
# 推論を実施するエンドポイント名を代入します
endpoint_name = "<ENDPOINT_NAME>"

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="./test.json",
)